### Analyse der Covid-19 Fälle in Deutschland

Datenquelle: Bereinigte Daten der Johns Hopkins Unviversity

In [95]:
import pandas as pd 
import plotly.graph_objects as go 
import math

In [96]:
def plot_layout(fig, Titel):
    fig.update_layout(
        titlefont = dict(size = 36),
        title= dict(text = Titel, y = 0.95, x = 0.5, xanchor = 'center'),
        hovermode = 'x',
        autosize = False,
        width=1600, height=800, paper_bgcolor='LightSteelBlue',
        legend = dict(x = 0.05, y = 0.95, font=dict(size=18, color="black"), bgcolor="lightgray")
        )
    fig.update_xaxes(tickfont= dict(size=18))
    fig.update_yaxes(tickfont= dict(size=18))
    return fig

In [97]:
df = pd.read_csv('https://raw.githubusercontent.com/Lucas-Czarnecki/COVID-19-CLEANED-JHUCSSE/master/COVID-19_CLEAN/csse_covid_19_time_series_cleaned/time_series_covid19_cases_tidy.csv',parse_dates=['Date'])

In [98]:
df.Date.iloc[-1]

Timestamp('2020-04-02 00:00:00')

In [99]:
df.rename(columns=dict(Country_Region = 'land', Confirmed = 'infiziert', Deaths = 'tot', Recovered = 'erholt', Date = 'datum'),inplace=True)

In [100]:
df = df[df.land == 'Germany']

In [101]:
# Berechnung von weiteren Daten auf der Basis der vorhandenen Informationen
df.sterberate = df.tot / df.infiziert * 100
df.aktiv = df.infiziert - df.tot - df.erholt
df.neu_infiziert = df.infiziert.diff()
df.proz_steig_infiziert = df.neu_infiziert / df.infiziert * 100
df.neu_aktiv = df.aktiv.diff()
df.proz_steig_aktiv = df.neu_aktiv / df.aktiv * 100
df.neu_tot = df.tot.diff()
df.proz_steig_tot = df.neu_tot / df.tot * 100
df.neu_erholt = df.erholt.diff()
df.proz_steig_erholt = df.neu_erholt / df.erholt * 100
df.beendet = df.erholt + df.tot
df.neu_beendet = df.beendet.diff()

In [102]:
df.tail()

,Province_State,land,Latitude,Longitude,datum,infiziert,tot,erholt
10291,NaN,Germany,51.0,9.0,2020-03-29,62095,533,9211.0
10292,NaN,Germany,51.0,9.0,2020-03-30,66885,645,13500.0
10293,NaN,Germany,51.0,9.0,2020-03-31,71808,775,16100.0
10294,NaN,Germany,51.0,9.0,2020-04-01,77872,920,18700.0
10295,NaN,Germany,51.0,9.0,2020-04-02,84794,1107,22440.0


Die Anzahl der Infizierten Personen in Deutschland ist abhängig von der Anzahl der getesteten Personen in Deutschland. Leider liegen aktuell keine verlässlichen Daten auf Tagesbasis für die Anzahl der getesteten Personen vor.

Bis zum 24.03. wurde nur als infiziert Fälle gezählt, die Positiv auf Corona Viren getestet wurden. Ab dem 25.03. werden auch Fälle als infiziert gezählt, bei denen Personen nicht getestet wurden, aber Kontakt zu positiv getesteten Personen hatten und Symptome zeigen.


In [103]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.beendet,mode='lines+markers', name='Beendete Infektionen',stackgroup='one'))
fig.add_trace(go.Scatter(x=df.datum, y=df.aktiv,mode='lines+markers',name='Aktive', fill='tonexty',stackgroup='one'))
fig.add_trace(go.Scatter(x=df.datum, y=df.infiziert,mode='lines+markers',name='Infiziert',))
fig = plot_layout(fig, 'Summe der Infizierten in Deutschland')
fig.show()

In [104]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.datum, y=df.neu_infiziert))
fig = plot_layout(fig, 'Tägliche Neuinfektionen')
fig.show()

In [105]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.proz_steig_infiziert,mode='lines+markers'))
fig = plot_layout(fig, 'Steigerungsrate Infizierte in %')
fig.show()


In [106]:
p = df.proz_steig_infiziert.iloc[-1]
q = 1 + p/100
t = math.log(2) / math.log(q)
print(f'Verdoppelung der infizierten Personen alle {t:.2f} Tage')

Verdoppelung der infizierten Personen alle 8.83 Tage


In [107]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.aktiv,mode='lines+markers'))
fig = plot_layout(fig, 'Summe der Aktiven Personen in Deutschland')
fig.show()

In [108]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.neu_beendet,mode='lines+markers', name='neu beendete Infektionen',stackgroup='one'))
fig.add_trace(go.Scatter(x=df.datum, y=df.neu_aktiv,mode='lines+markers',name='neu Aktive', fill='tonexty',stackgroup='one'))
fig.add_trace(go.Scatter(x=df.datum, y=df.neu_infiziert,mode='lines+markers',name='neu Infizierte',))
fig = plot_layout(fig, 'Neu Infizierte vs. Neu beendete Infektionen')
fig.show()

In [109]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.datum, y=df.neu_aktiv))
fig = plot_layout(fig, 'Tägliche neuen aktiven Fälle')
fig.show()

In [110]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.proz_steig_aktiv,mode='lines+markers'))
fig = plot_layout(fig, 'Steigerungsrate Aktive in %')
fig.show()

In [111]:
p = df.proz_steig_aktiv.iloc[-1]
q = 1 + p/100
t = math.log(2) / math.log(q)
print(f'Verdoppelung der aktiven Personen alle {t:.2f} Tage')

Verdoppelung der aktiven Personen alle 14.52 Tage


In [112]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.tot,mode='lines+markers'))
fig = plot_layout(fig, 'Summe der verstorbenen Personen in Deutschland')
fig.show()

In [113]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.datum, y=df.neu_tot))
fig = plot_layout(fig, 'Täglich Verstorbene')
fig.show()

In [114]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.proz_steig_tot,mode='lines+markers'))
fig = plot_layout(fig, 'Steigerungsrate Sterbefälle in %')
fig.show()

In Deutschland gilt jeder Verstorbene, bei dem Corona-Viren gefunden wurden als Corona Sterbefall, unabhängig davon, ob er an Corona verstorben ist.


In [115]:
print(f'Sterberate in Deutschland aktuell bei {df.sterberate.iloc[-1]:.2f} %')

Sterberate in Deutschland aktuell bei 1.31 %


In [116]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.erholt / df.beendet * 100,mode='lines+markers',name='erholt'))
fig.add_trace(go.Scatter(x=df.datum, y=df.tot / df.beendet * 100,mode='lines+markers', name='verstorben'))
fig = plot_layout(fig, 'Verteilung beendete Fälle in %')
fig.show()

# Sterblichkeit je Altersklassen

Leider gibt es aktuell keine verlässlichen Daten zur Sterblickheit in Deutschland je Altersklassen. Auf der Suche nach Daten bin ich schließlich auf Süd-Korea gestoßen. Süd-Korea ist insofern interessant für Deutschland, da der Zustand des Gesundheitssystem in etwas auf dem Stand von Deutschland ist und dort genaue Daten zur Corona-Krise verfügbar sind.

Lt. Bericht von Professor Kim Woo-joo des Guro-Krankenhauses der Korea-Universität vom 28.03.2020 (https://www.youtube.com/watch?v=gAk7aX5hksU&t=284s) liegt die Sterbewahrscheinlichkeit für die unterschiedlichen Altersgruppen ins Südkorea bei folgenden Werten:

0-9 Jahre 0%, 10 - 19 Jahre 0%, 20 - 29 Jahre 0%, 30 - 39 Jahre 0,1%, 40 - 49 Jahre 0,1%, 50 - 59 Jahre 0,5%, 60 - 69 Jahre 1,6%, 70 - 79 Jahre 6.3%, ab 80 Jahre 11.6%

Aus der Bevölerkerungspyramide Deutschland werden die Anzahl Personen je Altersklasse dagegengesetzt (in Tausend):
0-9 Jahre 7760, 10-19 Jahre 7537, 20-29 Jahre 9573, 30-39 Jahre 10940, 40-49 Jahre 10105, 50-59 Jahre 13331, 60-69 Jahre 10741, 70 - 79 Jahre 7456, 80-89 Jahre 5074, 90-99 Jahre 832

Bei sofortiger uneingeschränkter Bewegungsfreiheit ohne besonderen Schutz der gesamten Bevölkerung würden sich folgende Todesfälle ergeben


In [117]:
risikoklassen = dict(
    altersklasse = "0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99".split(','),
    anzahl =  [7760, 7537, 9573, 10940, 10105, 13331, 10741, 7456, 5074, 832],
    sterblichkeit = [0, 0, 0, 0.1, 0.1, 0.5, 1.6, 6.3, 11.6, 11.6])
df_alter = pd.DataFrame.from_dict(risikoklassen)
df_alter.sterbefälle = df_alter.anzahl * df_alter.sterblichkeit / 100 * 1000
fig = go.Figure()
fig.add_trace(go.Bar(x=df_alter.altersklasse, y=df_alter.sterbefälle))
fig = plot_layout(fig, 'Anzahl möglicher Tote je Altersklasse')
fig.show()